# 通过 langchain使用 faiss

In [ ]:
[langchain-faiss](https://python.langchain.com/docs/integrations/vectorstores/faiss/)

## 准备工作

In [1]:
%%time
%%capture

!pip install langchain
!pip install faiss-cpu
!pip install sentence_transformers

CPU times: user 31.1 ms, sys: 14 ms, total: 45.2 ms
Wall time: 7.46 s


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter

## 初始化BGE模型做嵌入编码

In [3]:
%%time

model_name = "/models/bge-large-zh-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

CPU times: user 2.73 s, sys: 810 ms, total: 3.54 s
Wall time: 3.17 s


In [4]:
%%time

embedding = embeddings.embed_query("hi this is harrison")
len(embedding)

CPU times: user 491 ms, sys: 20.3 ms, total: 512 ms
Wall time: 324 ms


1024

## 通过text加载器加载文本

In [5]:
%%time

loader = TextLoader("./faiss-langchain-content.txt")
documents = loader.load()

CPU times: user 1.15 ms, sys: 0 ns, total: 1.15 ms
Wall time: 2.5 ms


## 加入索引

In [6]:
%%time

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)
db.index.ntotal

CPU times: user 821 ms, sys: 25.7 ms, total: 846 ms
Wall time: 530 ms


1

## 搜索

In [7]:
query = "贝叶斯"
docs = db.similarity_search(query)

docs[0].page_content

'贝叶斯概率是一种概率解释的方式，它基于贝叶斯定理，用来描述在给定某些先验知识的情况下，随机事件的概率。它得名于 18 世纪的英国数学家 Thomas Bayes。'

## 转换为 Retriever

In [8]:
retriever = db.as_retriever()
docs = retriever.invoke(query)

docs[0].page_content

'贝叶斯概率是一种概率解释的方式，它基于贝叶斯定理，用来描述在给定某些先验知识的情况下，随机事件的概率。它得名于 18 世纪的英国数学家 Thomas Bayes。'

## 搜索并返回相似度分数

In [9]:
docs_and_scores = db.similarity_search_with_score(query)
docs_and_scores[0]

(Document(page_content='贝叶斯概率是一种概率解释的方式，它基于贝叶斯定理，用来描述在给定某些先验知识的情况下，随机事件的概率。它得名于 18 世纪的英国数学家 Thomas Bayes。', metadata={'source': './faiss-langchain-content.txt'}),
 0.99823606)

## 保存和加载 Faiss 索引

In [11]:
db.save_local("faiss-langchain_index")
new_db = FAISS.load_local("faiss-langchain_index", embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)

docs[0]

Document(page_content='贝叶斯概率是一种概率解释的方式，它基于贝叶斯定理，用来描述在给定某些先验知识的情况下，随机事件的概率。它得名于 18 世纪的英国数学家 Thomas Bayes。', metadata={'source': './faiss-langchain-content.txt'})

## TODO

- 序列化和反序列化 -- 使用数据库的时候需要
- 合并向量 -- 追加文档
- 带过滤的相似度搜索 -- 过滤基于文档属性，代码实现逻辑
- 删除存储记录
